In [12]:
import pandas as pd
df = pd.read_csv(r'data_banknote_authentication (1).txt',delimiter = ',',header = None)
df.columns = ['Variance','Skewness','Kurtosis','Entropy','Class']

In [15]:
df = df.sample(frac = 1)
df.reset_index(drop = True, inplace = True)

In [17]:
def gini_index(groups,classes):
    n_instances = float(sum([len(group) for group in groups]))
    gini = 0.0
    for group in groups:
        size = float(len(group))
        if size == 0:
            continue
        score = 0.0
        for class_category in classes:
            #classes in that group
            p = [row[-1] for row in group].count(class_category)/size
            score = score + p**2
        #gini score of the split
        #Weighted gini
        gini = gini + (1 - score)*(size/n_instances)
    return gini

In [18]:
#Creating the Split
#has the attribute of the dataset in concern and a value
#calculate gini index for the split
#split the dataset
#evaluate the splits

#producing splits means separating a dataset into 2 lists based on the index of
#an attribute and the split value for that attribute
#to split a dataset according to the CART algorithm, 

def test_split(index,value,dataset):
    left,right = list(),list()
    for row in dataset:
        if row[index]<value:
            left.append(row)
        else:
            right.append(row)
    return left,right

In [121]:
#Evaluating all the splits
#for a dataset we need to check each value on each attribute as a possible split
#evaluate the cost of the split and find the one with the lowest gini index close to zero
#We use a dictionary to represent a node in the decision tree as we can store data by name
#store the index of the chosen attribute
#store the value of the attribute to split by
#2 groups of the data split

def get_split(dataset):
    class_values = list(set(row[-1] for row in dataset))
    best_score = 9999
    for index in range(len(dataset[0]) - 1):
        for row in dataset:
            value = row[index]
            groups = test_split(index,value,dataset)
            gini = gini_index(groups,class_values)
            print('X{}<{},Gini:{}'.format(index+1,value,gini))
            if gini<best_score:
                best_index,best_value,best_score,best_group = index,value,gini,groups
    return {'index':best_index,'value':best_value,'groups':best_group,'gini':best_score}



In [122]:
from sklearn.datasets import make_moons
X,y = make_moons(n_samples = 100, noise = 0.15, random_state = 42)


In [123]:
import numpy as np
dataset = np.c_[X,y]
split = get_split(dataset)

X1<1.6138383345623006,Gini:0.4117647058823529
X1<0.08984723112298044,Gini:0.40808823529411764
X1<0.7472104652178698,Gini:0.42412778478352253
X1<-1.1017451405084246,Gini:0.4897959183673469
X1<-0.7287145541958098,Gini:0.4444444444444444
X1<-0.6589128125418795,Gini:0.43820224719101125
X1<0.2282826565833816,Gini:0.4527112232030265
X1<1.3943918380168097,Gini:0.375
X1<0.8715828348574295,Gini:0.41087344028520506
X1<0.8830447179010182,Gini:0.40027137042062416
X1<0.3822032643834266,Gini:0.4548374146928944
X1<-0.32279169270471836,Gini:0.3975903614457831
X1<1.2990062071841366,Gini:0.3589743589743589
X1<-0.635699742255501,Gini:0.4186046511627906
X1<1.968993869590418,Gini:0.46808510638297873
X1<1.2107329379402356,Gini:0.3824
X1<1.1399819127545174,Gini:0.3742203742203742
X1<1.0680515924312788,Gini:0.3799603174603175
X1<0.4959857130688907,Gini:0.4549819927971188
X1<-0.26038714962243836,Gini:0.375
X1<-0.06516218662264421,Gini:0.3421052631578947
X1<1.8700584460980159,Gini:0.45054945054945056
X1<0.03146

In [31]:
split

{'index': 1,
 'value': 0.02650645155129651,
 'groups': ([array([ 1.61383833, -0.49115086,  1.        ]),
   array([ 0.74721047, -0.36911116,  1.        ]),
   array([-1.10174514,  0.23685641,  0.        ]),
   array([-0.72871455,  0.14652347,  0.        ]),
   array([ 1.39439184, -0.45063627,  1.        ]),
   array([ 0.88304472, -0.12658445,  0.        ]),
   array([ 1.29900621, -0.64914275,  1.        ]),
   array([1.96899387, 0.297549  , 1.        ]),
   array([1.21073294, 0.37721455, 0.        ]),
   array([ 1.13998191, -0.36088456,  1.        ]),
   array([ 1.06805159, -0.53460667,  1.        ]),
   array([-0.06516219,  0.13092506,  1.        ]),
   array([ 1.87005845, -0.18659309,  1.        ]),
   array([-1.03151461,  0.35788726,  0.        ]),
   array([0.45814234, 0.00308109, 1.        ]),
   array([2.02675758, 0.09006383, 1.        ]),
   array([ 0.71349755, -0.61055519,  1.        ]),
   array([-1.05805492,  0.1152524 ,  0.        ]),
   array([ 1.80904221, -0.49580332,  1. 

For now we know how to do a single split on a dataset
How to build a tree?
To build the root node, we call the get_split() function for the entire dataset


When to stop growing a tree:
- Maximum Tree Depth is reached
- Minimum Node Records (The minimum number of training patterns that a given node is responsible for)

A **Terminal Node** is a node where predictions are made based on the highest class of instances which occur in it.

In [40]:
#create a terminal node value
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    count_dict = {outcome:outcomes.count(outcome) for outcome in outcomes}
    return max(count_dict.items(), key = lambda x:x[1])[0]

Recursive Splitting

Once a node is created, create child nodes recursively on this by calling the same function until we hit the required conditions of a terminal node.

In [124]:
def recursive_split(node,max_depth,min_size,depth):
    left,right = node['groups']
    del(node['groups'])
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    if depth >= max_depth:
        node['left'] = to_terminal(left)
        node['right'] = to_terminal(right)
        return
    #processing left split
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    #if all the above conditions are not met, we can split this group 
    else:
        node['left'] = get_split(left)
        recursive_split(node['left'],max_depth,min_size,depth+1)
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        recursive_split(node['right'],max_depth,min_size,depth+1)

In [125]:
def build_tree(train,max_depth,min_size):
    #first split
    root = get_split(train)
    #start tree
    recursive_split(root,max_depth,min_size,1)
    return root

In [132]:
def print_tree(tree,depth = 0):
    if isinstance(tree,dict):
        print('{}[X{}<{}]'.format(depth*' ',tree['index'],tree['value']))
        print_tree(tree['left'],depth + 1)
        print_tree(tree['right'],depth + 1)
    else:
        print('{}[{}]'.format(depth*' ',tree))

In [134]:
tree = build_tree(dataset,3,1)
print_tree(tree)
tree

X1<1.6138383345623006,Gini:0.4117647058823529
X1<0.08984723112298044,Gini:0.40808823529411764
X1<0.7472104652178698,Gini:0.42412778478352253
X1<-1.1017451405084246,Gini:0.4897959183673469
X1<-0.7287145541958098,Gini:0.4444444444444444
X1<-0.6589128125418795,Gini:0.43820224719101125
X1<0.2282826565833816,Gini:0.4527112232030265
X1<1.3943918380168097,Gini:0.375
X1<0.8715828348574295,Gini:0.41087344028520506
X1<0.8830447179010182,Gini:0.40027137042062416
X1<0.3822032643834266,Gini:0.4548374146928944
X1<-0.32279169270471836,Gini:0.3975903614457831
X1<1.2990062071841366,Gini:0.3589743589743589
X1<-0.635699742255501,Gini:0.4186046511627906
X1<1.968993869590418,Gini:0.46808510638297873
X1<1.2107329379402356,Gini:0.3824
X1<1.1399819127545174,Gini:0.3742203742203742
X1<1.0680515924312788,Gini:0.3799603174603175
X1<0.4959857130688907,Gini:0.4549819927971188
X1<-0.26038714962243836,Gini:0.375
X1<-0.06516218662264421,Gini:0.3421052631578947
X1<1.8700584460980159,Gini:0.45054945054945056
X1<0.03146

{'index': 1,
 'value': 0.02650645155129651,
 'gini': 0.24253888188314426,
 'left': {'index': 0,
  'value': -0.03394273799800426,
  'gini': 0.04993252361673399,
  'left': 0.0,
  'right': {'index': 1,
   'value': -0.12658445405158886,
   'gini': 0.047846889952153145,
   'left': 1.0,
   'right': 1.0}},
 'right': {'index': 0,
  'value': 1.572588036143677,
  'gini': 0.17486338797814208,
  'left': {'index': 1,
   'value': 0.7085288830916046,
   'gini': 0.1728395061728395,
   'left': 0.0,
   'right': 0.0},
  'right': {'index': 0,
   'value': 1.968993869590418,
   'gini': 0.0,
   'left': 1.0,
   'right': 1.0}}}

In [135]:
def predict_tree(tree,row):
    if (row[tree['index']] < tree['value']):
        if isinstance(tree['left'],dict):
            return predict_tree(tree['left'],row)
        else:
            return tree['left']
    else:
        if isinstance(tree['right'],dict):
            return predict_tree(tree['right'],row)
        else:
            return tree['right']

In [137]:
res = list()
for row in X:
    res.append(predict_tree(tree,row))

np.c_[np.array(res),y]

array([[1., 1.],
       [0., 1.],
       [1., 1.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [1., 1.],
       [0., 0.],
       [1., 0.],
       [0., 0.],
       [0., 0.],
       [1., 1.],
       [0., 0.],
       [1., 1.],
       [0., 0.],
       [1., 1.],
       [1., 1.],
       [0., 0.],
       [0., 0.],
       [0., 1.],
       [1., 1.],
       [0., 0.],
       [0., 0.],
       [1., 1.],
       [0., 0.],
       [0., 0.],
       [1., 1.],
       [0., 0.],
       [1., 1.],
       [0., 0.],
       [0., 0.],
       [1., 1.],
       [0., 0.],
       [1., 1.],
       [0., 0.],
       [1., 1.],
       [1., 1.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [1., 1.],
       [0., 0.],
       [0., 0.],
       [1., 1.],
       [0., 0.],
       [1., 1.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [1., 1.],
       [0., 0.],
       [1., 1.],
       [0., 0.